instructions: For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.


Bonus point: Solving this problem using pandas (instead of any other approach)

In [2]:
#importar pandas y numpy y matplot y  quede embebido  el notebook

import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [8]:
#leer datos bookings

bt = pd.read_csv('bookings.csv.bz2', sep='^',nrows=1e5)

In [9]:
bt.head(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [6]:
b.columns

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')

In [30]:
#en la realidad hay que hacer un exploratorio
#uy hay espacios 

b = pd.read_csv('bookings.csv.bz2',
                sep='^',
                nrows=1e5,
                usecols=['brd_time           ','brd_port','off_port'],
                parse_dates =['brd_time           '] 
               )

In [11]:
b.head()

,brd_port,off_port,brd_time
0,LHR,ZRH,2013-03-07 08:50:00
1,SAL,CLT,2013-04-12 13:04:00
2,CLT,SAL,2013-07-15 07:00:00
3,AKL,SVO,2013-04-24 23:59:00
4,SVO,AKL,2013-05-14 20:15:00


In [31]:
b['date'] = b['brd_time           ']

In [32]:
b.head(3)

,brd_port,off_port,brd_time,date
0,LHR,ZRH,2013-03-07 08:50:00,2013-03-07 08:50:00
1,SAL,CLT,2013-04-12 13:04:00,2013-04-12 13:04:00
2,CLT,SAL,2013-07-15 07:00:00,2013-07-15 07:00:00


In [33]:
#borrar columna repetida

b = b.drop('brd_time           ',axis=1)

In [16]:
#leer searches
st = pd.read_csv('searches.csv.bz2',
                sep='^',
                nrows=1e5,
                )

In [17]:
st.head(3)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC


In [18]:
st.columns

Index([u'Date', u'Time', u'TxnCode', u'OfficeID', u'Country', u'Origin',
       u'Destination', u'RoundTrip', u'NbSegments', u'Seg1Departure',
       u'Seg1Arrival', u'Seg1Date', u'Seg1Carrier', u'Seg1BookingCode',
       u'Seg2Departure', u'Seg2Arrival', u'Seg2Date', u'Seg2Carrier',
       u'Seg2BookingCode', u'Seg3Departure', u'Seg3Arrival', u'Seg3Date',
       u'Seg3Carrier', u'Seg3BookingCode', u'Seg4Departure', u'Seg4Arrival',
       u'Seg4Date', u'Seg4Carrier', u'Seg4BookingCode', u'Seg5Departure',
       u'Seg5Arrival', u'Seg5Date', u'Seg5Carrier', u'Seg5BookingCode',
       u'Seg6Departure', u'Seg6Arrival', u'Seg6Date', u'Seg6Carrier',
       u'Seg6BookingCode', u'From', u'IsPublishedForNeg', u'IsFromInternet',
       u'IsFromVista', u'TerminalID', u'InternetOffice'],
      dtype='object')

In [34]:
#selecciono las columnas que quiero --> ojo quito time de bookings

s = pd.read_csv('searches.csv.bz2',
                sep='^',
                nrows=1e5,
                usecols=['Seg1Date','Origin','Destination'],
                parse_dates =['Seg1Date'] 
               )

In [35]:
s.head(5)

,Origin,Destination,Seg1Date
0,TXL,AUH,2013-01-26
1,ATH,MIL,2013-01-04
2,ICT,SFO,2013-08-02
3,RNB,ARN,2013-01-02
4,OSL,MAD,2013-03-22


In [36]:
#Unir ambos  b y d
s.__class__

pandas.core.frame.DataFrame

In [37]:
b.__class__

pandas.core.frame.DataFrame

In [42]:
b.date[1]

Timestamp('2013-04-12 13:04:00')

In [44]:
s.Seg1Date[1]

Timestamp('2013-01-04 00:00:00')

In [51]:
#crear columnas sin horas

b['dateNoTime'] = b.date.apply(lambda d: d.date())

In [53]:
s['dateNoTime'] = s.Seg1Date.apply(lambda d: d.date())

In [54]:
s.head(5)

,Origin,Destination,Seg1Date,dateNoTime
0,TXL,AUH,2013-01-26,2013-01-26
1,ATH,MIL,2013-01-04,2013-01-04
2,ICT,SFO,2013-08-02,2013-08-02
3,RNB,ARN,2013-01-02,2013-01-02
4,OSL,MAD,2013-03-22,2013-03-22


In [55]:
b.head(5)

,brd_port,off_port,date,dateNoTime
0,LHR,ZRH,2013-03-07 08:50:00,2013-03-07
1,SAL,CLT,2013-04-12 13:04:00,2013-04-12
2,CLT,SAL,2013-07-15 07:00:00,2013-07-15
3,AKL,SVO,2013-04-24 23:59:00,2013-04-24
4,SVO,AKL,2013-05-14 20:15:00,2013-05-14


In [57]:
#merge (hay que decidir cual derecha, cuál izquierda y que dimensiones) 
#iner  solo va a quedarte con lo que coincide, pierdas cuántos no machean
# left --> izq búsquedas y derecha reservas

m = pd.merge(s,
             b,
             how='left',
             left_on=['Origin','Destination','dateNoTime'],
             right_on=['brd_port','off_port','dateNoTime'],
            )

In [58]:
m.head(5)

,Origin,Destination,Seg1Date,dateNoTime,brd_port,off_port,date
0,TXL,AUH,2013-01-26,2013-01-26,NaN,NaN,NaT
1,ATH,MIL,2013-01-04,2013-01-04,NaN,NaN,NaT
2,ICT,SFO,2013-08-02,2013-08-02,NaN,NaN,NaT
3,RNB,ARN,2013-01-02,2013-01-02,NaN,NaN,NaT
4,OSL,MAD,2013-03-22,2013-03-22,NaN,NaN,NaT


In [59]:
#creo columna para ver si hay coincidencia

m['match'] = m.brd_port.apply(lambda b:not pd.isnull(b))

In [60]:
m.head()

,Origin,Destination,Seg1Date,dateNoTime,brd_port,off_port,date,match
0,TXL,AUH,2013-01-26,2013-01-26,NaN,NaN,NaT,False
1,ATH,MIL,2013-01-04,2013-01-04,NaN,NaN,NaT,False
2,ICT,SFO,2013-08-02,2013-08-02,NaN,NaN,NaT,False
3,RNB,ARN,2013-01-02,2013-01-02,NaN,NaN,NaT,False
4,OSL,MAD,2013-03-22,2013-03-22,NaN,NaN,NaT,False


In [61]:
m.groupby('match').count()

,Origin,Destination,Seg1Date,dateNoTime,brd_port,off_port,date
match,,,,,,,
False,100000,100000,99605,100000,0,0,0


In [ ]:
#seg1Date --> No hace match porque hay espacios y porque Destination son ciudades y no aeropuertos

